# Correlation Power Analysis (Brier et al. 2004) - Evolution Plot


#### Learning goals
- Learn how to detect how many traces are needed for a CPA attack
- Learn what an Evolution Plot is

#### References
- `lascar.ScoreProgressionOutputMethod`

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import lascar
import numpy as np
import plotly.graph_objects as pgo

from securec.capture import capture

## How many traces are needed?

To detect the minimal number of required traces for a CPA attack we have to determine when the _correct_ key guess is distinguishable from all _incorrect_ ones.

An _Evolution Plot_ can visualize this.

In [ ]:
class EvolutionOutputMethod(lascar.ScoreProgressionOutputMethod):
    def _finalize(self):
        scores = np.array(list(self.scores.values())[0]).T
        fig = pgo.Figure()
        for guess, score in zip(self.engines[0]._guess_range, scores):
            fig.add_trace(pgo.Scatter(y=score, name=str(guess)))
        fig.show()

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 1</div>

Create an Evolution Plot using Lascar's `CpaEngine`.

Hints:
- `lascar.Session` takes an argument to store intermediate values in regular steps. Append `output_steps=range(0, len(data), 100)` to the list of parameters when creating the session.

</div>

In [ ]:
data = capture(
    platform="elmo",
    number_of_traces=500,
    number_of_samples=0,
    fromfile=os.path.abspath("../lecture_3/sbox_lookup.c"),
    inputfunction=lambda _: [random.randint(0, 255)] + 15 * [0],
)

In [ ]:
def selection_function(value, guess):
    return lascar.hamming(lascar.tools.aes.sbox[value["input"][0] ^ guess])

trace = lascar.TraceBatchContainer(data["trace"], data)
engine = lascar.CpaEngine(
    selection_function=selection_function,
    guess_range=range(256),
)

session = lascar.Session(
    trace,
    engine=engine,
    output_method=EvolutionOutputMethod(engine),
    output_steps=range(0, len(data), 100),
)
session.run(batch_size="auto")